<a href="https://colab.research.google.com/github/ryorod/MMMFT/blob/master/colab/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMMFT API mock server
### https://mmmft.ryorod.com/

In [ ]:
#@title CONFIGURE AWS

!pip install awscli
!aws configure

In [ ]:
#@title INSTALL APP

%cd /content
!git clone https://github.com/ryorod/MMMFT.git
%cd MMMFT/api_for_colab
%pip install -r requirements.txt
!cp -R app/* /content

import sys
sys.path.append('/content/MMMFT/api/app/utils')

In [ ]:
#@title INSTALL RESOURCES

from tqdm import tqdm
from google.colab import output
pbar = tqdm(total=5)

def update(x):
  pbar.set_description(x)
  output.clear()
  pbar.update(1)

%cd /content
update("downloading source code")
!rm -rf MMM_API &> /dev/null
!rm -rf MMM_API_SEP_2.zip &> /dev/null
!wget https://www.sfu.ca/~jeffe/MMM_API_SEP_2.zip &> /dev/null
!unzip -o MMM_API_SEP_2.zip &> /dev/null

update("installing dependencies")
!apt-get install libprotobuf-dev protobuf-compiler &> /dev/null
!wget https://download.pytorch.org/libtorch/cu102/libtorch-cxx11-abi-shared-with-deps-1.8.1%2Bcu102.zip &> /dev/null
!unzip -o libtorch-cxx11-abi-shared-with-deps-1.8.1+cu102.zip &> /dev/null

update("installing mmm-api (takes a few minutes)")
%cd /content/MMM_API
!bash build_python_google.sh --google &> /dev/null
%cd python
%pip install . &> /dev/null
import mmm_api as mmm

update("installing python modules")
!pip install pyFluidSynth &> /dev/null
!apt install fluidsynth &> /dev/null
!pip install midi2audio &> /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2 &> /dev/null

update("downloading model")
%cd /content
!wget -O model.pt http://vault.sfu.ca/index.php/s/Ff2j19IrKS5R969/download &> /dev/null


In [ ]:
#@title RUN APP

import nest_asyncio
from pyngrok import ngrok
import uvicorn

from app import app

if __name__ == '__main__':
  ngrok_tunnel = ngrok.connect(9999)
  print('Public URL:', ngrok_tunnel.public_url)
  nest_asyncio.apply()
  uvicorn.run(app, port=9999)

references

https://colab.research.google.com/drive/10ZAdEwHDbL1lVcUGeCdj9FxXnQSNFSH4?usp=sharing  
https://stackoverflow.com/questions/63833593/how-to-run-fastapi-uvicorn-in-google-colab